# Where should the Next Mid-Ohio Foodbank Panty Be Locatated? 

Outline:
- Import panty location data from ____
- Import customer location data from pt_families
- Import census data (identify potential customers with qualifying - actual)
- Plot a map of central Ohio
- Write county lines on Ohio map
- Plot pantry location on Ohio map
- Plot existing customers on Ohio map
- Show proportions of qualifying vs actual customers on county map

In [1]:
import pandas as pd
import folium
import os
from geopy.geocoders import Nominatim
os.getcwd()

'C:\\Users\\jorda\\OneDrive\\Documents\\GitHub\\data-io-2019'

## Import Data

### Import Ohio Foodbank Pantry Locations

In [2]:
pantry_df = pd.read_csv('food-bank-loc/locations.csv')
pantry_df.head()

,loc_id,loc_num,org_id,parent_id,loc_name,loc_nickname,address1,address2,city,state,...,sc_monthly_20,sc_monthly_25,sc_monthly_30,sc_monthly_35,sc_monthly_40,date_added,added_by,last_update,last_update_by,status_id
0,1,1351,1,0,Lutheran Social Services ChoicePantry - South ...,LSS Champion,1460 S CHAMPION AVE,NaN,COLUMBUS,OH,...,"120,159,254,219,281,308,265,278,291,297,379,36...","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0",17/2/2012 05:47:27,1,18/10/2019 12:35:27,1,1
1,2,1354,1,0,Lutheran Social Services ChoicePantry - West C...,LSS West,NaN,NaN,COLUMBUS,OH,...,"78,70,67,108,96,97,83,115,107,151,167,163,50","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0",21/6/2012 16:00:21,1,16/10/2019 14:20:16,1036,1
2,3,1476,1,0,Lutheran Social Services ChoicePantry - Caldwe...,LSS Caldwell,315 MAIN ST,NaN,CALDWELL,OH,...,"101,137,134,121,122,112,100,69,90,92,82,100,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0",17/7/2012 14:58:29,1,15/10/2019 19:34:56,5791,1
3,4,1339,1,0,Lutheran Social Services ChoicePantry - Fairfi...,LSS Lancaster,2045 E MAIN ST,NaN,LANCASTER,OH,...,"151,202,201,137,258,168,157,202,205,227,195,20...","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,1,1,0,0,5,1,0,0,1,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0",17/2/2012 05:57:17,1,18/10/2019 10:18:39,5791,1
4,6,1521,2,0,Mid-Ohio Foodbank - Kroger Community Pantry,MOF Kroger Pantry,3960 BROOKHAM DR,NaN,GROVE CITY,OH,...,"737,898,988,710,848,1289,1697,732,668,924,808,...","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,1,1,0,1,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0","0,0,0,0,0,0,0,0,0,0,0,0,0",17/2/2012 06:00:08,1,19/10/2019 08:08:11,596,1


In [3]:
pantry_df.columns

Index(['loc_id', 'loc_num', 'org_id', 'parent_id', 'loc_name', 'loc_nickname',
       'address1', 'address2', 'city', 'state',
       ...
       'sc_monthly_20', 'sc_monthly_25', 'sc_monthly_30', 'sc_monthly_35',
       'sc_monthly_40', 'date_added', 'added_by', 'last_update',
       'last_update_by', 'status_id'],
      dtype='object', length=103)

In [4]:
pantry_loc_df = pantry_df[['loc_id', 'loc_num','loc_nickname',
                          'address1', 'city', 'state', 'zip',
                          'county', 'latitude', 'longitude',
                          'pt_latitude', 'pt_longitude',
                          'census_block', 'census_tract']]
pantry_loc_df

,loc_id,loc_num,loc_nickname,address1,city,state,zip,county,latitude,longitude,pt_latitude,pt_longitude,census_block,census_tract
0,1,1351,LSS Champion,1460 S CHAMPION AVE,COLUMBUS,OH,43206,FRANKLIN,39.93534,-82.97080,39.934993,-82.971104,3011,59.00
1,2,1354,LSS West,NaN,COLUMBUS,OH,43204,FRANKLIN,39.95535,-83.09418,39.955354,-83.094183,2065,82.10
2,3,1476,LSS Caldwell,315 MAIN ST,CALDWELL,OH,43724,NOBLE,39.74678,-81.51509,39.746782,-81.515159,3052,9684.00
3,4,1339,LSS Lancaster,2045 E MAIN ST,LANCASTER,OH,43130,FAIRFIELD,39.71569,-82.55336,39.716011,-82.553450,1026,313.00
4,6,1521,MOF Kroger Pantry,3960 BROOKHAM DR,GROVE CITY,OH,43123,FRANKLIN,39.88132,-83.05145,39.886235,-83.055387,4003,97.20
5,7,1015,Grove City Pantry,2710 COLUMBUS ST,GROVE CITY,OH,43123,FRANKLIN,39.88162,-83.07623,39.881411,-83.075302,2005,97.11
6,8,1096,Gladden Pantry,318 S GLENWOOD AVE,COLUMBUS,OH,43223,FRANKLIN,39.95375,-83.02911,39.952104,-83.029036,2020,50.00
7,10,1231,Southside Comm Min Pantry,225 E GATES ST,COLUMBUS,OH,43206,FRANKLIN,39.93738,-82.98953,39.937388,-82.989537,3001,58.10
8,11,1510,Far East Feeding Ministry,3160 Brice Road,BRICE,OH,43109,FRANKLIN,0.00000,0.00000,39.913745,-82.832508,2058,93.74
9,13,4170,East Linden Elementary,2505 BRENTNELL AVE,COLUMBUS,OH,43211,FRANKLIN,40.02500,-82.94792,40.025009,-82.947929,1015,75.20


In [5]:
pantry_latlong = pantry_loc_df[['loc_nickname', 'latitude', 'longitude']]
pantry_latlong

,loc_nickname,latitude,longitude
0,LSS Champion,39.93534,-82.97080
1,LSS West,39.95535,-83.09418
2,LSS Caldwell,39.74678,-81.51509
3,LSS Lancaster,39.71569,-82.55336
4,MOF Kroger Pantry,39.88132,-83.05145
5,Grove City Pantry,39.88162,-83.07623
6,Gladden Pantry,39.95375,-83.02911
7,Southside Comm Min Pantry,39.93738,-82.98953
8,Far East Feeding Ministry,0.00000,0.00000
9,East Linden Elementary,40.02500,-82.94792


### Create a Map of Central Ohio

In [6]:
# Get the coordinates for the City of Columbus (for folium)
address = "Columbus"
geolocator = Nominatim(user_agent="columbus")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates for the City of Columbus are {}, {}.'.format(latitude, longitude))

The coordinates for the City of Columbus are 39.9622601, -83.0007065.


In [7]:
# Create a folium map around the coordinates of the city
map_columbus = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# Create a tags on each pantry
for pantry_name, lat, long in zip(pantry_latlong['loc_nickname'], pantry_latlong['latitude'], pantry_latlong['longitude']):
    label = pantry_name
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, long],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_opacity=0.1,
                        parse_html=False).add_to(map_columbus)  

map_columbus